# 1. PROCESO CREACION DATAFRAMES DIVIDIDO POR GDP

Importamos librerias

In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os 
import missingno as msn 
from fuzzywuzzy import process

import warnings
warnings.filterwarnings('ignore')

c:\Users\javier.pina\AppData\Local\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Leo los archivos

In [3]:
Vacunacion = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\global_vaccination_coverage.csv')
Ratio_suicidio = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\death_rate_from_suicides_gho.csv')
esperanza_vida = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\life_expectancy.csv')
causa_muerte = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\annual_number_of_deaths_by_cause.csv')
ratio_homicidio = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\homicide_rate_unodc.csv')
pobreza = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\poverty_explorer.csv')

# Los cargo pero son los filtros que usaremos mas adelante

gdp_per_capita = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\gdp_per_capita_penn_world_table.csv')
gasto_en_salud = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\public_healthcare_spending_share_gdp.csv')


In [4]:
# elimino entity de mis df que no sean esperanza de vida para que luego no me den problemas, excepto de pobreza

Vacunacion = Vacunacion.drop(columns='Entity')
Ratio_suicidio = Ratio_suicidio.drop(columns='Entity')
causa_muerte = causa_muerte.drop(columns='Entity')
ratio_homicidio = ratio_homicidio.drop(columns='Entity')


In [5]:
# Creo los merges para cada tabla con esperanza de vida y luego con PIB

# Unimos con esperanza de vida

combinada_VN = pd.merge(esperanza_vida, Vacunacion, on=['Code', 'Year'], how='left')
combinada_RS = pd.merge(esperanza_vida, Ratio_suicidio, on=['Code', 'Year'], how='left')
combinada_CM = pd.merge(esperanza_vida, causa_muerte, on=['Code', 'Year'], how='left')
combinada_RH = pd.merge(esperanza_vida, ratio_homicidio, on=['Code', 'Year'], how='left')
combinada_PB = pd.merge(esperanza_vida, pobreza, right_on=['Country', 'Year'],left_on=['Entity', 'Year'],)
combinada_PIB = pd.merge(esperanza_vida, gasto_en_salud, on=['Code', 'Year'], how='left')


In [6]:
# Ponemos que todos sean mayores a los 2000

combinada_VN = combinada_VN[combinada_VN['Year'] >= 2000]
combinada_RS = combinada_RS[combinada_RS['Year'] >= 2000]
combinada_CM = combinada_CM[combinada_CM['Year'] >= 2000]
combinada_RH = combinada_RH[combinada_RH['Year'] >= 2000]
combinada_PB = combinada_PB[combinada_PB['Year'] >= 2000]
combinada_PIB = combinada_PIB[combinada_PIB['Year'] >= 2000]

In [7]:
# Creamos los 5 quintiles

gdp_per_capita.rename(columns= {"GDP per capita (output, multiple price benchmarks)": "Renta per capita" }, inplace= True)
df_agrupado_renta_per_capita = gdp_per_capita.groupby(['Entity', 'Code'], as_index=False)['Renta per capita'].mean()
df_agrupado_renta_per_capita['Quintil'] = pd.qcut(df_agrupado_renta_per_capita['Renta per capita'], q=5, labels=['Q1Renta', 'Q2Renta', 'Q3Renta', 'Q4Renta', 'Q5Renta'])
df_agrupado_renta_per_capita = df_agrupado_renta_per_capita.drop(columns='Entity')

In [8]:
# Uno los datframes con los quintiles 

Tabla_definitiva_VN = pd.merge(combinada_VN, df_agrupado_renta_per_capita, on='Code', how='left')
Tabla_definitiva_RS = pd.merge(combinada_RS, df_agrupado_renta_per_capita, on='Code', how='left')
Tabla_definitiva_CM = pd.merge(combinada_CM, df_agrupado_renta_per_capita, on='Code', how='left')
Tabla_definitiva_RH = pd.merge(combinada_RH, df_agrupado_renta_per_capita, on='Code', how='left')
Tabla_definitiva_PB = pd.merge(combinada_PB, df_agrupado_renta_per_capita, on='Code', how='left')
Tabla_definitiva_PIB = pd.merge(combinada_PIB, df_agrupado_renta_per_capita, on='Code', how='left')

Tabla_definitiva_CM

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,Deaths - Meningitis - Sex: Both - Age: All Ages (Number),Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number),Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number),Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number),Deaths - Malaria - Sex: Both - Age: All Ages (Number),Deaths - Drowning - Sex: Both - Age: All Ages (Number),...,Deaths - Poisonings - Sex: Both - Age: All Ages (Number),Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number),Deaths - Road injuries - Sex: Both - Age: All Ages (Number),Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number),Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number),Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number),"Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)",Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number),Renta per capita,Quintil
0,Afghanistan,AFG,2000,55.2978,3105.0,1291.0,416.0,3060.0,263.0,2048.0,...,499.0,3013.0,6581.0,7471.0,3249.0,6206.0,465.0,4334.0,NaN,NaN
1,Afghanistan,AFG,2001,55.7981,3063.0,1300.0,424.0,2973.0,211.0,2044.0,...,502.0,2924.0,6720.0,7565.0,3267.0,6277.0,467.0,4363.0,NaN,NaN
2,Afghanistan,AFG,2002,56.4538,2990.0,1311.0,429.0,2795.0,2883.0,1980.0,...,490.0,2746.0,6696.0,7455.0,3238.0,6246.0,455.0,4251.0,NaN,NaN
3,Afghanistan,AFG,2003,57.3445,3300.0,1326.0,434.0,3039.0,2188.0,2282.0,...,531.0,2984.0,7243.0,7471.0,3342.0,6443.0,499.0,4373.0,NaN,NaN
4,Afghanistan,AFG,2004,57.9436,3350.0,1344.0,439.0,3033.0,773.0,2386.0,...,547.0,2978.0,7522.0,7477.0,3394.0,6541.0,516.0,4345.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,Zimbabwe,ZWE,2017,60.7095,1460.0,781.0,223.0,2990.0,2116.0,818.0,...,398.0,2962.0,2473.0,2818.0,2007.0,4342.0,654.0,144.0,3241.8753,Q2Renta
14450,Zimbabwe,ZWE,2018,61.4141,1450.0,795.0,227.0,2918.0,2088.0,825.0,...,400.0,2890.0,2509.0,2849.0,2030.0,4377.0,657.0,139.0,3241.8753,Q2Renta
14451,Zimbabwe,ZWE,2019,61.2925,1450.0,812.0,232.0,2884.0,2068.0,827.0,...,405.0,2855.0,2554.0,2891.0,2065.0,4437.0,662.0,136.0,3241.8753,Q2Renta
14452,Zimbabwe,ZWE,2020,61.1242,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3241.8753,Q2Renta


# 2. PROCESO CREACION DATAFRAMES DIVIDIDO POR PIB

In [9]:
# Creamos los 5 quintiles para PIB

df_no_nulos_gasto_salud = gasto_en_salud[gasto_en_salud['Code'].notnull()]
df_no_nulos_gasto_salud.rename(columns={'Domestic general government health expenditure (% of GDP)': '%PIB'}, inplace=True)
df_agrupado_gasto_salud = df_no_nulos_gasto_salud.groupby(['Entity', 'Code'], as_index=False)['%PIB'].mean()
df_agrupado_gasto_salud['Quintil'] = pd.qcut(df_agrupado_gasto_salud['%PIB'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
quintiles, bins = pd.qcut(df_agrupado_gasto_salud['%PIB'], q=5, labels=['Q1','Q2','Q3','Q4','Q5'], retbins=True)
df_agrupado_gasto_salud = df_agrupado_gasto_salud.drop(columns= 'Entity')

In [10]:
# Uno los datframes con los quintiles 

Tabla_definitiva2_VN = pd.merge(combinada_VN, df_agrupado_gasto_salud, on='Code', how='left')
Tabla_definitiva2_RS = pd.merge(combinada_RS, df_agrupado_gasto_salud, on='Code', how='left')
Tabla_definitiva2_CM = pd.merge(combinada_CM, df_agrupado_gasto_salud, on='Code', how='left')
Tabla_definitiva2_RH = pd.merge(combinada_RH, df_agrupado_gasto_salud, on='Code', how='left')
Tabla_definitiva2_PB = pd.merge(combinada_PB, df_agrupado_gasto_salud, on='Code', how='left')
Tabla_definitiva2_PIB = pd.merge(combinada_PIB, df_agrupado_gasto_salud, on='Code', how='left')

Tabla_definitiva2_CM

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,Deaths - Meningitis - Sex: Both - Age: All Ages (Number),Deaths - Alzheimer's disease and other dementias - Sex: Both - Age: All Ages (Number),Deaths - Parkinson's disease - Sex: Both - Age: All Ages (Number),Deaths - Nutritional deficiencies - Sex: Both - Age: All Ages (Number),Deaths - Malaria - Sex: Both - Age: All Ages (Number),Deaths - Drowning - Sex: Both - Age: All Ages (Number),...,Deaths - Poisonings - Sex: Both - Age: All Ages (Number),Deaths - Protein-energy malnutrition - Sex: Both - Age: All Ages (Number),Deaths - Road injuries - Sex: Both - Age: All Ages (Number),Deaths - Chronic respiratory diseases - Sex: Both - Age: All Ages (Number),Deaths - Cirrhosis and other chronic liver diseases - Sex: Both - Age: All Ages (Number),Deaths - Digestive diseases - Sex: Both - Age: All Ages (Number),"Deaths - Fire, heat, and hot substances - Sex: Both - Age: All Ages (Number)",Deaths - Acute hepatitis - Sex: Both - Age: All Ages (Number),%PIB,Quintil
0,Afghanistan,AFG,2000,55.2978,3105.0,1291.0,416.0,3060.0,263.0,2048.0,...,499.0,3013.0,6581.0,7471.0,3249.0,6206.0,465.0,4334.0,0.535133,Q1
1,Afghanistan,AFG,2001,55.7981,3063.0,1300.0,424.0,2973.0,211.0,2044.0,...,502.0,2924.0,6720.0,7565.0,3267.0,6277.0,467.0,4363.0,0.535133,Q1
2,Afghanistan,AFG,2002,56.4538,2990.0,1311.0,429.0,2795.0,2883.0,1980.0,...,490.0,2746.0,6696.0,7455.0,3238.0,6246.0,455.0,4251.0,0.535133,Q1
3,Afghanistan,AFG,2003,57.3445,3300.0,1326.0,434.0,3039.0,2188.0,2282.0,...,531.0,2984.0,7243.0,7471.0,3342.0,6443.0,499.0,4373.0,0.535133,Q1
4,Afghanistan,AFG,2004,57.9436,3350.0,1344.0,439.0,3033.0,773.0,2386.0,...,547.0,2978.0,7522.0,7477.0,3394.0,6541.0,516.0,4345.0,0.535133,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,Zimbabwe,ZWE,2017,60.7095,1460.0,781.0,223.0,2990.0,2116.0,818.0,...,398.0,2962.0,2473.0,2818.0,2007.0,4342.0,654.0,144.0,2.089709,Q2
14450,Zimbabwe,ZWE,2018,61.4141,1450.0,795.0,227.0,2918.0,2088.0,825.0,...,400.0,2890.0,2509.0,2849.0,2030.0,4377.0,657.0,139.0,2.089709,Q2
14451,Zimbabwe,ZWE,2019,61.2925,1450.0,812.0,232.0,2884.0,2068.0,827.0,...,405.0,2855.0,2554.0,2891.0,2065.0,4437.0,662.0,136.0,2.089709,Q2
14452,Zimbabwe,ZWE,2020,61.1242,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.089709,Q2


In [ ]:
# Descargo los EXCELS GDP

Tabla_definitiva_VN.to_excel('GDP_Vacunacion.xlsx', index= False) 
Tabla_definitiva_RS.to_excel('GDP_Suicidios.xlsx', index= False) 
Tabla_definitiva_CM.to_excel('GDP_Causa_muerte.xlsx', index= False) 
Tabla_definitiva_RH.to_excel('GDP_Homicidios.xlsx', index= False) 
Tabla_definitiva_PB.to_excel('GDP_Pobreza.xlsx', index= False) 
Tabla_definitiva_PIB.to_excel('GDP_PIB.xlsx', index= False) 

# Descargo los EXCELS PIB

Tabla_definitiva2_VN.to_excel('PIB_Vacunacion.xlsx', index= False) 
Tabla_definitiva2_RS.to_excel('PIB_Suicidios.xlsx', index= False) 
Tabla_definitiva2_CM.to_excel('PIB_Causa_muerte.xlsx', index= False) 
Tabla_definitiva2_RH.to_excel('PIB_Homicidios.xlsx', index= False) 
Tabla_definitiva2_PB.to_excel('PIB_Pobreza.xlsx', index= False) 
Tabla_definitiva2_PIB.to_excel('PIB_GDP.xlsx', index= False) 

In [15]:

Tabla_definitiva3_VN = pd.merge(Tabla_definitiva_VN, Tabla_definitiva2_VN, on=['Code', 'Year'], how='left')

Tabla_definitiva3_VN

,Entity_x,Code,Year,Period life expectancy at birth - Sex: all - Age: 0_x,BCG (% of one-year-olds immunized)_x,HepB3 (% of one-year-olds immunized)_x,Hib3 (% of one-year-olds immunized)_x,IPV1 (% of one-year-olds immunized)_x,MCV1 (% of one-year-olds immunized)_x,PCV3 (% of one-year-olds immunized)_x,...,IPV1 (% of one-year-olds immunized)_y,MCV1 (% of one-year-olds immunized)_y,PCV3 (% of one-year-olds immunized)_y,Pol3 (% of one-year-olds immunized)_y,RCV1 (% of one-year-olds immunized)_y,RotaC (% of one-year-olds immunized)_y,YFV (% of one-year-olds immunized)_y,DTP3 (% of one-year-olds immunized)_y,%PIB,Quintil_y
0,Afghanistan,AFG,2000,55.2978,30.0,NaN,NaN,NaN,27.0,NaN,...,NaN,27.0,NaN,24.0,NaN,NaN,NaN,24.0,0.535133,Q1
1,Afghanistan,AFG,2001,55.7981,43.0,NaN,NaN,NaN,37.0,NaN,...,NaN,37.0,NaN,35.0,NaN,NaN,NaN,33.0,0.535133,Q1
2,Afghanistan,AFG,2002,56.4538,46.0,NaN,NaN,NaN,35.0,NaN,...,NaN,35.0,NaN,36.0,NaN,NaN,NaN,36.0,0.535133,Q1
3,Afghanistan,AFG,2003,57.3445,44.0,NaN,NaN,NaN,39.0,NaN,...,NaN,39.0,NaN,41.0,NaN,NaN,NaN,41.0,0.535133,Q1
4,Afghanistan,AFG,2004,57.9436,51.0,NaN,NaN,NaN,48.0,NaN,...,NaN,48.0,NaN,50.0,NaN,NaN,NaN,50.0,0.535133,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322009,Zimbabwe,ZWE,2017,60.7095,95.0,89.0,89.0,NaN,90.0,89.0,...,NaN,90.0,89.0,89.0,90.0,91.0,NaN,89.0,2.089709,Q2
322010,Zimbabwe,ZWE,2018,61.4141,95.0,89.0,89.0,NaN,88.0,89.0,...,NaN,88.0,89.0,89.0,88.0,90.0,NaN,89.0,2.089709,Q2
322011,Zimbabwe,ZWE,2019,61.2925,95.0,90.0,90.0,65.0,85.0,90.0,...,65.0,85.0,90.0,90.0,85.0,92.0,NaN,90.0,2.089709,Q2
322012,Zimbabwe,ZWE,2020,61.1242,88.0,86.0,86.0,86.0,85.0,86.0,...,86.0,85.0,86.0,86.0,85.0,88.0,NaN,86.0,2.089709,Q2


# CREACION DE DATOS POR TIPO DE DATAFRAME INDIVIDUAL

In [3]:
Vacunacion2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\global_vaccination_coverage.csv')
Ratio_suicidio2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\death_rate_from_suicides_gho.csv')
esperanza_vida2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\life_expectancy.csv')
causa_muerte2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\annual_number_of_deaths_by_cause.csv')
ratio_homicidio2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\homicide_rate_unodc.csv')
pobreza2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\poverty_explorer.csv')

# Los cargo pero son los filtros que usaremos mas adelante

gdp_per_capita2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\gdp_per_capita_penn_world_table.csv')
gasto_en_salud2 = pd.read_csv(r'C:\Users\javier.pina\Desktop\PLAN PIÑA\NUCLIO\PROYECTO FINAL\public_healthcare_spending_share_gdp.csv')


In [4]:
# Limito a mas del 2000

Vacunacion2 = Vacunacion2[Vacunacion2['Year'] >= 2000]
ratio_homicidio2 = ratio_homicidio2[ratio_homicidio2['Year'] >= 2000]
esperanza_vida2 = esperanza_vida2[esperanza_vida2['Year'] >= 2000]
causa_muerte2 = causa_muerte2[causa_muerte2['Year'] >= 2000]
Ratio_suicidio2 = Ratio_suicidio2[Ratio_suicidio2['Year'] >= 2000]
pobreza1 = pobreza2[pobreza2['Year'] >= 2000]
gasto_en_salud2 = gasto_en_salud2[gasto_en_salud2['Year'] >= 2000]
gdp_per_capita2 = gdp_per_capita2[gdp_per_capita2['Year'] >= 2000]

In [7]:
Vacunacion2['ID'] = Vacunacion2['Entity'].astype(str) + Vacunacion2['Year'].astype(str)
Ratio_suicidio2['ID'] = Ratio_suicidio2['Entity'].astype(str) + Ratio_suicidio2['Year'].astype(str)
esperanza_vida2['ID'] = esperanza_vida2['Entity'].astype(str) + esperanza_vida2['Year'].astype(str)
causa_muerte2['ID'] = causa_muerte2['Entity'].astype(str) + causa_muerte2['Year'].astype(str)
ratio_homicidio2['ID'] = ratio_homicidio2['Entity'].astype(str) + ratio_homicidio2['Year'].astype(str)
pobreza2['ID'] = pobreza2['Country'].astype(str) + pobreza2['Year'].astype(str)
gasto_en_salud2['ID'] = gasto_en_salud2['Entity'].astype(str) + gasto_en_salud2['Year'].astype(str)
gdp_per_capita2['ID'] = gdp_per_capita2['Entity'].astype(str) + gdp_per_capita2['Year'].astype(str)

gasto_en_salud2

,Entity,Code,Year,Domestic general government health expenditure (% of GDP),ID
0,Afghanistan,AFG,2002,0.084181,Afghanistan2002
1,Afghanistan,AFG,2003,0.650963,Afghanistan2003
2,Afghanistan,AFG,2004,0.542926,Afghanistan2004
3,Afghanistan,AFG,2005,0.529184,Afghanistan2005
4,Afghanistan,AFG,2006,0.497840,Afghanistan2006
...,...,...,...,...,...
4009,Zimbabwe,ZWE,2015,1.551368,Zimbabwe2015
4010,Zimbabwe,ZWE,2016,1.788909,Zimbabwe2016
4011,Zimbabwe,ZWE,2017,1.794181,Zimbabwe2017
4012,Zimbabwe,ZWE,2018,2.782935,Zimbabwe2018


In [31]:
Vacunacion2.to_excel('Vacunacion.xlsx', index= False) 
Ratio_suicidio2.to_excel('Suicidio.xlsx', index= False) 
esperanza_vida2.to_excel('Esperanza_vida.xlsx', index= False) 
causa_muerte2.to_excel('Causa_muerte.xlsx', index= False) 
ratio_homicidio2.to_excel('Homicidio.xlsx', index= False) 
pobreza2.to_excel('Pobreza_definitivo.xlsx', index= False) 
gdp_per_capita2.to_excel('GDP_Definitivo.xlsx', index= False) 
gasto_en_salud2.to_excel('Gasto_PIB_definitivo.xlsx', index= False) 